# Sample on how to use modelmesh-serving with SDK v1beta1 python

## This notebook shows how to use the python sdk with ModelMesh to create an InferenceService

In [1]:
from kubernetes import client
from kserve import constants
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec
from kserve import V1beta1StorageSpec
from kserve import KServeClient

### Define the global variables to be used in InferenceService creation

In [2]:
namespace = 'modelmesh-serving'
name='msnist-sample'
protocol_version='v2'
api_version = constants.KSERVE_V1BETA1

### Define InferenceService with the proper ModelMesh annotation

In [3]:
isvc = V1beta1InferenceService(
    api_version=constants.KSERVE_V1BETA1,
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=name, 
        namespace=namespace,
        annotations={
            'serving.kserve.io/deploymentMode': 'ModelMesh'
        }
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            sklearn=V1beta1SKLearnSpec(
                protocol_version=protocol_version,
                storage=V1beta1StorageSpec(
                    key='localMinIO',
                    path='sklearn/mnist-svm.joblib'
                )
            )
        )
    )
)

### Create InferenceService

In [4]:
kserve = KServeClient()
kserve.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'serving.kserve.io/deploymentMode': 'ModelMesh'},
  'creationTimestamp': '2023-05-24T00:01:41Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:serving.kserve.io/deploymentMode': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:sklearn': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:storage': {'.': {}, 'f:key': {}, 'f:path': {}}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-05-24T00:01:41Z'}],
  'name': 'msnist-sample',
  'namespace': 'modelmesh-serving',
  'resourceVersion': '3402353',
  'uid': 'e6415d63-a57f-4540-ab17-d91cbc4da865'},
 'spec': {'predictor': {'sklearn': {'name': '',
    'protocolVersion': 'v2',
    'storage': {'key': 'localMinIO', 'path': 'skle

### Check to see if InferenceService is ready

In [5]:
kserve.wait_isvc_ready(name, namespace=namespace)

In [6]:
is_ready = kserve.is_isvc_ready(name, namespace=namespace)

In [7]:
print(is_ready)

True
